# MySQL 연동 기본 통합 실습

이 노트북은 PyMySQL을 사용한 MySQL 데이터베이스 연동에 대한 종합적인 학습 자료입니다.

## 목차
1. [PyMySQL 환경 설정](#1.-PyMySQL-환경-설정)
2. [기본 데이터베이스 연결](#2.-기본-데이터베이스-연결)
3. [데이터 조회 방법들](#3.-데이터-조회-방법들)
4. [튜플 vs 딕셔너리 커서](#4.-튜플-vs-딕셔너리-커서)
5. [데이터 삽입과 트랜잭션](#5.-데이터-삽입과-트랜잭션)
6. [기본 CRUD 예제](#6.-기본-CRUD-예제)
7. [완전한 방명록 시스템](#7.-완전한-방명록-시스템)
8. [실무 활용 팁과 주의사항](#8.-실무-활용-팁과-주의사항)

---


## 1. PyMySQL 환경 설정

### PyMySQL이란?
PyMySQL은 Python에서 MySQL 데이터베이스에 접속하기 위한 라이브러리입니다.
- 순수 Python으로 작성되어 설치가 간편
- MySQL 5.1 이상 버전 지원
- DB-API 2.0 표준을 준수

### 설치 방법


In [ ]:
# PyMySQL 설치 (터미널에서 실행)
# pip install pymysql

# 라이브러리 import
import pymysql
from typing import Dict, List, Tuple, Any

print("PyMySQL 버전:", pymysql.__version__)
print("설치 완료!")


## 2. 기본 데이터베이스 연결

### 연결 방법
MySQL 데이터베이스에 연결하기 위해서는 다음 정보가 필요합니다:
- **host**: 서버 IP 주소 (localhost 또는 127.0.0.1)
- **user**: 데이터베이스 사용자 계정
- **password**: 사용자 패스워드
- **db**: 연결할 데이터베이스명
- **port**: MySQL 포트번호 (기본값: 3306)


In [ ]:
# 데이터베이스 연결 설정
conn = pymysql.connect(
    host='localhost',      # 서버 IP
    user='user01',         # 계정 아이디  
    password='1234',       # 패스워드
    db='mydb',            # 데이터베이스명
    port=3306             # 포트번호
)

# 커서 생성 (데이터베이스 작업을 위한 객체)
curs = conn.cursor()

print("데이터베이스 연결 성공!")
print("연결 정보:", conn.get_server_info())


## 3. 데이터 조회 방법들

PyMySQL에서는 세 가지 주요 데이터 조회 방법을 제공합니다:
- **fetchall()**: 모든 레코드를 한번에 가져오기
- **fetchone()**: 첫 번째 레코드 하나만 가져오기  
- **fetchmany(size)**: 지정한 개수만큼 가져오기


In [ ]:
# 1. fetchall() - 모든 데이터 가져오기
sql = "SELECT * FROM guestbook"
curs.execute(sql)
rows = curs.fetchall()  # 튜플 타입으로 반환

print("=== fetchall() 결과 ===")
for row in rows:
    print(type(row), row)


In [ ]:
# 2. fetchone() - 한 개만 가져오기
print("\n=== fetchone() 결과 ===")
curs.execute(sql)
row = curs.fetchone()  # 첫 번째 레코드 하나만
print("첫 번째 레코드:", row)


In [ ]:
# 3. fetchmany() - 지정한 개수만 가져오기
print("\n=== fetchmany(3) 결과 ===")
curs.execute(sql)
rows = curs.fetchmany(3)  # 3개의 레코드만 가져오기
for row in rows:
    print(type(row), row)


## 4. 튜플 vs 딕셔너리 커서

### 기본 커서 (튜플 타입)
- 데이터를 튜플 형태로 반환
- 인덱스 번호로 접근 (row[0], row[1], ...)
- 메모리 효율적

### 딕셔너리 커서 (Dict 타입)  
- 데이터를 딕셔너리 형태로 반환
- 컬럼명으로 접근 (row['title'], row['contents'], ...)
- 가독성이 좋음


In [ ]:
# 딕셔너리 커서로 변경
curs = conn.cursor(pymysql.cursors.DictCursor)

sql = "SELECT * FROM guestbook"
curs.execute(sql)
rows = curs.fetchall()

print("=== 딕셔너리 커서 결과 ===")
for row in rows:
    # 컬럼명으로 직접 접근 가능
    print(f"제목: {row['title']}, 내용: {row['contents']}, 작성일: {row['wdate']}")
    print(f"타입: {type(row)}")
    break  # 첫 번째 행만 출력


## 5. 데이터 삽입과 트랜잭션

### 트랜잭션이란?
- 하나 이상의 쿼리가 하나의 목적을 위해 움직일 때
- 하나가 오류가 생기면 다른 하나도 취소시켜야 함
- **commit**: 확정 (변경사항을 데이터베이스에 반영)
- **rollback**: 되돌리기 (변경사항을 취소)

### DML과 커밋
- DML (Data Manipulation Language): INSERT, UPDATE, DELETE
- DML 작업 후에는 반드시 `commit()`을 호출해야 함


In [ ]:
# 데이터 삽입 예제
sql = """
    INSERT INTO guestbook(title, contents, writer, wdate)
    VALUES(%s, %s, %s, now())
"""

# 파라미터를 사용한 안전한 데이터 삽입
curs.execute(sql, ('제목6', '내용6', '작성자6'))

# 반드시 커밋해야 데이터베이스에 반영됨
conn.commit()

print("데이터 삽입 완료!")

# 삽입된 데이터 확인
sql = "SELECT * FROM guestbook ORDER BY id DESC LIMIT 1"
curs.execute(sql)
result = curs.fetchone()
print("방금 삽입된 데이터:", result)


## 6. 기본 CRUD 예제

CRUD는 데이터베이스의 기본 작업을 의미합니다:
- **C**reate: 생성 (INSERT)
- **R**ead: 조회 (SELECT)  
- **U**pdate: 수정 (UPDATE)
- **D**elete: 삭제 (DELETE)


In [ ]:
# 1. CREATE (생성) - INSERT
def insert_data():
    sql = """
        INSERT INTO guestbook(title, contents, writer, wdate)
        VALUES(%s, %s, %s, now())
    """
    curs.execute(sql, ('테스트 제목', '테스트 내용', '테스트 작성자'))
    conn.commit()
    print("✅ 데이터 생성 완료")

# 2. READ (조회) - SELECT  
def read_data():
    sql = "SELECT * FROM guestbook"
    curs.execute(sql)
    rows = curs.fetchall()
    print("📖 전체 데이터 조회:")
    for row in rows:
        print(f"  ID: {row[0]}, 제목: {row[1]}")

# 3. UPDATE (수정)
def update_data(guestbook_id: int):
    sql = """
        UPDATE guestbook 
        SET title = %s, contents = %s, writer = %s 
        WHERE id = %s
    """
    curs.execute(sql, ('수정된 제목', '수정된 내용', '수정된 작성자', guestbook_id))
    conn.commit()
    print(f"✏️ ID {guestbook_id} 데이터 수정 완료")

# 4. DELETE (삭제)
def delete_data(guestbook_id: int):
    sql = "DELETE FROM guestbook WHERE id = %s"
    curs.execute(sql, (guestbook_id,))
    conn.commit()
    print(f"🗑️ ID {guestbook_id} 데이터 삭제 완료")

# CRUD 작업 실행 예제
print("=== CRUD 작업 예제 ===")
insert_data()
read_data()


## 7. 완전한 방명록 시스템

이제 실제로 동작하는 완전한 방명록 시스템을 만들어보겠습니다.
사용자 입력을 받아 CRUD 작업을 수행하는 콘솔 애플리케이션입니다.


In [ ]:
# 완전한 방명록 시스템
class GuestbookSystem:
    def __init__(self, connection):
        self.conn = connection
        self.curs = connection.cursor(pymysql.cursors.DictCursor)
    
    def insert(self):
        """새 방명록 작성"""
        title = input("제목: ")
        writer = input("작성자: ")
        contents = input("내용: ")
        
        sql = """
            INSERT INTO guestbook(title, contents, writer, wdate)
            VALUES (%s, %s, %s, now())
        """
        self.curs.execute(sql, (title, contents, writer))
        self.conn.commit()
        print("✅ 방명록이 등록되었습니다!")
    
    def update(self):
        """방명록 수정"""
        guestbook_id = input("수정할 아이디: ")
        title = input("수정할 제목: ")
        writer = input("수정할 작성자: ")
        contents = input("수정할 내용: ")
        
        sql = """
            UPDATE guestbook
            SET title = %s, contents = %s, writer = %s
            WHERE id = %s
        """
        self.curs.execute(sql, (title, contents, writer, guestbook_id))
        self.conn.commit()
        print("✏️ 방명록이 수정되었습니다!")
    
    def delete(self):
        """방명록 삭제"""
        guestbook_id = input("삭제할 아이디: ")
        
        sql = "DELETE FROM guestbook WHERE id = %s"
        self.curs.execute(sql, (guestbook_id,))
        self.conn.commit()
        print("🗑️ 방명록이 삭제되었습니다!")
    
    def output(self):
        """방명록 목록 조회"""
        sql = "SELECT * FROM guestbook ORDER BY id DESC"
        self.curs.execute(sql)
        rows = self.curs.fetchall()
        
        print("\\n" + "="*50)
        print("📋 방명록 목록")
        print("="*50)
        
        for row in rows:
            print(f"ID: {row['id']}")
            print(f"제목: {row['title']}")
            print(f"작성자: {row['writer']}")
            print(f"내용: {row['contents']}")
            print(f"작성일: {row['wdate']}")
            print("-" * 30)
    
    def run(self):
        """메인 실행 루프"""
        print("🎉 방명록 시스템에 오신 것을 환영합니다!")
        
        while True:
            print("\\n" + "="*30)
            sel = input("1.목록 2.추가 3.수정 4.삭제 0.종료: ")
            
            if sel == "1":
                self.output()
            elif sel == "2":
                self.insert()
            elif sel == "3":
                self.update()
            elif sel == "4":
                self.delete()
            elif sel == "0":
                print("👋 방명록 시스템을 종료합니다.")
                break
            else:
                print("❌ 올바른 번호를 입력해주세요.")

# 방명록 시스템 생성
guestbook = GuestbookSystem(conn)

# 데모용으로 목록만 보여주기 (실제 실행은 주석 해제)
print("=== 방명록 시스템 데모 ===")
guestbook.output()

# 실제 방명록 시스템 실행하려면 아래 주석을 해제하세요
# guestbook.run()


## 8. 실무 활용 팁과 주의사항

### 🔒 보안 관련
1. **SQL 인젝션 방지**: 항상 파라미터를 사용하세요
2. **연결 정보 보호**: 패스워드를 코드에 하드코딩하지 마세요
3. **권한 최소화**: 필요한 권한만 부여하세요

### ⚡ 성능 최적화
1. **연결 재사용**: 매번 새 연결을 만들지 마세요
2. **커넥션 풀 활용**: 다중 사용자 환경에서는 커넥션 풀 사용
3. **적절한 인덱스 활용**: 자주 조회하는 컬럼에 인덱스 생성

### 🛠️ 에러 처리
1. **try-except 사용**: 데이터베이스 에러 처리
2. **롤백 처리**: 에러 발생 시 롤백 수행
3. **연결 해제**: finally 블록에서 연결 해제

### 📝 코딩 스타일
1. **함수 분리**: 각 기능을 별도 함수로 분리
2. **클래스 활용**: 관련 기능을 클래스로 묶기
3. **타입 힌트**: 함수 매개변수에 타입 명시


In [ ]:
# 실무 스타일의 데이터베이스 클래스
class DatabaseManager:
    def __init__(self, host: str, user: str, password: str, database: str, port: int = 3306):
        self.connection_config = {
            'host': host,
            'user': user, 
            'password': password,
            'db': database,
            'port': port
        }
        self.conn = None
        self.connect()
    
    def connect(self):
        """데이터베이스 연결"""
        try:
            self.conn = pymysql.connect(**self.connection_config)
            print("✅ 데이터베이스 연결 성공")
        except Exception as e:
            print(f"❌ 데이터베이스 연결 실패: {e}")
    
    def execute_query(self, sql: str, params: tuple = None, fetch: str = 'all'):
        """쿼리 실행 (조회용)"""
        try:
            with self.conn.cursor(pymysql.cursors.DictCursor) as curs:
                curs.execute(sql, params)
                
                if fetch == 'all':
                    return curs.fetchall()
                elif fetch == 'one':
                    return curs.fetchone()
                elif fetch == 'many':
                    return curs.fetchmany(5)
                    
        except Exception as e:
            print(f"❌ 쿼리 실행 오류: {e}")
            return None
    
    def execute_dml(self, sql: str, params: tuple = None):
        """DML 실행 (INSERT, UPDATE, DELETE)"""
        try:
            with self.conn.cursor() as curs:
                curs.execute(sql, params)
                self.conn.commit()
                return True
        except Exception as e:
            print(f"❌ DML 실행 오류: {e}")
            self.conn.rollback()
            return False
    
    def close(self):
        """연결 해제"""
        if self.conn:
            self.conn.close()
            print("🔌 데이터베이스 연결 해제")

# 실무 스타일 사용 예제
print("=== 실무 스타일 데이터베이스 클래스 ===")

# 데이터베이스 매니저 생성
db = DatabaseManager(
    host='localhost',
    user='user01',
    password='1234',
    database='mydb'
)

# 안전한 데이터 조회
result = db.execute_query("SELECT * FROM guestbook LIMIT 3")
if result:
    print("📊 조회 결과:")
    for row in result:
        print(f"  - {row}")

# 안전한 데이터 삽입
success = db.execute_dml(
    "INSERT INTO guestbook(title, contents, writer, wdate) VALUES(%s, %s, %s, now())",
    ('실무 예제', '실무 스타일 코드', '개발자')
)

if success:
    print("✅ 데이터 삽입 성공")
else:
    print("❌ 데이터 삽입 실패")

# 연결 해제는 프로그램 종료 시에만
# db.close()


---

## 실습 총정리

이 노트북에서 학습한 내용:

### 🎯 핵심 개념
1. **PyMySQL 기본 사용법**: 연결, 커서, 쿼리 실행
2. **데이터 조회 방법**: fetchall(), fetchone(), fetchmany()
3. **커서 타입**: 튜플 vs 딕셔너리
4. **트랜잭션 관리**: commit(), rollback()
5. **CRUD 작업**: Create, Read, Update, Delete

### 💡 주요 학습 포인트
- **파라미터 사용**: SQL 인젝션 방지를 위한 안전한 쿼리
- **딕셔너리 커서**: 가독성 높은 데이터 접근
- **예외 처리**: try-except를 통한 안전한 프로그래밍
- **클래스 활용**: 코드 재사용성과 유지보수성 향상

### 🚀 다음 단계 학습 제안
- **ORM 학습**: SQLAlchemy, Django ORM
- **비동기 프로그래밍**: aiomysql, asyncio
- **웹 프레임워크 연동**: Flask, Django, FastAPI
- **데이터베이스 설계**: 정규화, 인덱스 최적화

### 📚 추가 자료
- [PyMySQL 공식 문서](https://pymysql.readthedocs.io/)
- [MySQL 공식 문서](https://dev.mysql.com/doc/)
- [SQL 튜토리얼](https://www.w3schools.com/sql/)

---

### 연결 해제
```python
# 작업 완료 후 연결 해제
conn.close()
print("🎉 MySQL 연동 기본 실습 완료!")
```

*MySQL 연동 기본 통합 실습 완료*


In [ ]:
# 작업 완료 후 연결 해제
conn.close()
print("🎉 MySQL 연동 기본 실습 완료!")
